# Via google drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Obtenção da base de dados

In [ ]:
%%time
!git clone https://github.com/Diegoomal/ebeer_dataset.git

# Instalação de pacotes extras

In [ ]:
# ! pip install imutils --quiet
# ! pip install adabelief-tf --quiet
# ! pip install mlflow --quiet

# Importação dos modulos necessários

In [3]:
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mlflow

import tensorflow as tf
tf.config.run_functions_eagerly(True)

from adabelief_tf import AdaBeliefOptimizer
from sklearn.model_selection import train_test_split

from datetime import datetime
import random as python_random
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Input, AveragePooling2D)
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler

ModuleNotFoundError: No module named 'cv2'

# Configuração do MLflow

In [ ]:
# MLFLOW_TRACKING_USERNAME = 'diego.maldonado'
# MLFLOW_TRACKING_PASSWORD = 'd431b7b196e28028b386d497062f5ecad26855ed'
# MLFLOW_TRACKING_URI = 'https://dagshub.com/diego.maldonado/proj_integrado_cnn_cerveja.mlflow'

# os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME
# os.environ['MLFLOW_TRACKING_PASSWORD'] = MLFLOW_TRACKING_PASSWORD

# mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# mlflow.tensorflow.autolog(log_models=True, log_input_examples=True, log_model_signatures=True)

# Definição de funções adicionais

In [ ]:
def read_data_files(root, folder, sub_folder, file_name):
  with open(os.path.join(root, folder, sub_folder, file_name)) as f:
    files_names = [line.replace('\n','') for line in f.readlines()]
  return files_names

def read_files_names(root, folder):
  return os.listdir(os.path.join(root, folder))

def read_image(root, folder, file_name) -> None:
 return tf.keras.utils.load_img(
      os.path.join(root, folder, file_name),
      grayscale=False,
      color_mode='rgb',
      target_size=None,
      interpolation='nearest'
  )

def reset_seeds():
   np.random.seed(123) 
   python_random.seed(123)
   tf.random.set_seed(1234)

# Leitura do dataset e atribuição às respectivas variáveis

In [ ]:
width = 128
height = 128
channels = 3

size = (height, width)

In [ ]:
datasetPath = '/src/ebeer_dataset/'

In [ ]:
# filePath = 'skol_lata_350ml'

# filesNames = read_files_names(datasetPath, filePath)

# train_data_skol = [np.array(image).astype('float32')/255 for image in [read_image(datasetPath, filePath, image_name).resize(size) for image_name in filesNames]]


In [ ]:
# filePath = 'brahma_duplo_malte_lata_350ml'

# filesNames = read_files_names(datasetPath, filePath)

# train_data_brahma = [np.array(image).astype('float32')/255 for image in [read_image(datasetPath, filePath, image_name).resize(size) for image_name in filesNames]]


In [ ]:
# filePath = 'heiniken_lata_350ml'

# filesNames = read_files_names(datasetPath, filePath)

# train_data_heiniken = [np.array(image).astype('float32')/255 for image in [read_image(datasetPath, filePath, image_name).resize(size) for image_name in filesNames]]


In [ ]:
# escapes = [".git", "tests", "README.md"]

# for index, currentpath in enumerate(os.listdir(datasetPath), start=0):

#   # data not interest dir's for training
#   if (escapes.__contains__(currentpath)):
#     continue

#   print(index+1, '/', len(os.listdir(datasetPath)), '- diretorio:', currentpath)

#   # meatada.json

#   # verify if contains 'metadata.json' file
#   path_json = os.path.join(datasetPath, currentpath, "metadata.json")
#   if (os.path.exists(path_json)):
#     print("metada exists")
#   else:
#     break

#   # images

#   path_imgs = os.path.join(datasetPath, currentpath)
#   filesname = read_files_names(datasetPath, path_imgs)

#   # verify if contains images at dir
#   if (len(filesname) > 0):
#     print("images exists")
#   elif (len(filesname) == 0):
#     print("images not exists")
#     break

## data augmentation

In [ ]:
import os
import PIL
import random

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import skimage
print(skimage.__version__)

ModuleNotFoundError: No module named 'skimage'

In [ ]:
from skimage import io, img_as_ubyte
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise

In [ ]:
def rotation_anti_clockwise(image):
  angle = random.randint(0, 180)
  return rotate(image, angle)

def rotation_clockwise(image):
  angle = random.randint(0, 180)
  return rotate(image, -angle)

def h_flip(image):
  return np.fliplr(image)

def v_flip(image):
  return np.flipud(image)

def brightness(image):
  bright = np.ones(image.shape, dtype="uint8") * 70
  brightincrease = cv2.add(image, bright)
  return brightincrease

def blur_img(image):
  k_size = random.randrange(1, 10, 2)
  img_blur = cv2.medianBlur(image, k_size)
  return img_blur

def noise_img(image):
  return random_noise(image)

def zoom(image):
  zoom_value = random.random()
  hidth, width = image.shape[:2]
  h_taken = int(zoom_value*hidth)
  w_taken = int(zoom_value*width)
  h_start = random.randint(0, hidth-h_taken)
  w_start = random.randint(0, width-w_taken)
  image = image[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
  image = cv2.resize(image, (hidth, width), cv2.INTER_CUBIC)
  return image

In [ ]:
transformations = {
  'rotation_anti_clockwise': rotation_anti_clockwise,
  'rotation_clockwise': rotation_clockwise,
  'brightness': brightness,
  'blur_img': blur_img,
  'noise': noise_img,
  'h_flip': h_flip,
  'v_flip': v_flip,
  'zoom': zoom
}

In [ ]:
%%time

import datetime

escapes = [".git", "tests", "README.md"]

for index, currentpath in enumerate(os.listdir(datasetPath), start=0):

  # data not interest dir's for training
  if (escapes.__contains__(currentpath)):
    continue

  print(index+1, '/', len(os.listdir(datasetPath)), '- diretorio:', currentpath)

  filesname = read_files_names(datasetPath, currentpath)
  filesname.remove("metadata.json")

  # images = [read_image(datasetPath, currentpath, image_name) for image_name in filesname]
  # print('len(images):', len(images))

  for i, filename in enumerate(filesname, start=0):
    for j, transformation in enumerate(list(transformations), start=0):

      image_source = read_image(datasetPath, currentpath, filename) 

      image_transformed = transformations[transformation](image_source)
      image_transformed = img_as_ubyte(image_transformed)
      image_transformed = cv2.cvtColor(image_transformed, cv2.COLOR_BGR2RGB)

      cv2.imwrite(
          os.path.join(
              datasetPath, 
              currentpath,
              str(datetime.datetime.now().timestamp()).replace('.', '_') + ".jpg"
          ),
          image_transformed
      )

## dados disponiveis para treino

In [ ]:
# %%time

# import json
# import datetime

# n_neuron_out = 0

# metadatas = []
# escapes = [".git", "tests", "README.md"]

# for index, currentpath in enumerate(os.listdir(datasetPath), start=0):

#   # data not interest dir's for training
#   if (escapes.__contains__(currentpath)):
#     continue

#   print(index+1, '/', len(os.listdir(datasetPath)), '- diretorio:', currentpath)

#   # meatada.json

#   # verify if contains 'metadata.json' file
#   path_json = os.path.join(datasetPath, currentpath, "metadata.json")
#   if (os.path.exists(path_json)):
#     with open(path_json) as f:
#         data = json.load(f)
#         metadatas.append(data)
#   else:
#     break

#   # images

#   # print('datasetPath:', datasetPath, '- currentpath:', currentpath)
#   # path_images = os.path.join(datasetPath, currentpath)
#   # print(path_images)

#   filesname = read_files_names(datasetPath, currentpath)
#   filesname.remove("metadata.json")
#   # print(filesname)

#   # # verify if contains images at dir
#   # if (len(filesname) == 0):
#   #   break

#   images = [read_image(datasetPath, currentpath, image_name) for image_name in filesname]
#   # print(len(images))

#   # data_augemntation
#   for i, image_source in enumerate(images, start=0):
#     for j, transformation in enumerate(list(transformations), start=0):

#       image_transformed = transformations[transformation](image_source)
#       image_transformed = img_as_ubyte(image_transformed)
#       image_transformed = cv2.cvtColor(image_transformed, cv2.COLOR_BGR2RGB)

#       cv2.imwrite(
#           os.path.join(
#               path_imgs,
#               str(datetime.datetime.now().timestamp()).replace('.', '_') + ".jpg"
#           ),
#           image_transformed
#       )

#   # # neurons for out of CNN
#   n_neuron_out+=1


# Separação do dataset em treino e teste

In [ ]:
n_neuronios_saida = 20

labels_index = {    
     0: "amstel"
  ,  1: "bavaria"
  ,  2: "bohemia"
  ,  3: "brahma"
  ,  4: "colorado_appia_garrafa"
  ,  5: "colorado_appia_lata"
  ,  6: "colorado_indica_garrafa"
  ,  7: "colorado_indica_lata"
  ,  8: "coronita_long_neck"
  ,  9: "heiniken"
  , 10: "imperio"
  , 11: "kaiser"
  , 12: "original"
  , 13: "skol_beats_gt"
  , 14: "skol_beats_secret"
  , 15: "skol_beats_senses"
  , 16: "skol"
  , 17: "sol"
  , 18: "spaten"
  , 19: "stella_artrois"
}

In [ ]:
train_data_X = np.concatenate(
    (
        train_data_amstel
        , train_data_bavaria
        , train_data_bohemia
        , train_data_brahma
        , train_data_colorado_appia_garrafa
        , train_data_colorado_appia_lata
        , train_data_colorado_indica_garrafa
        , train_data_colorado_indica_lata
        , train_data_coronita_long_neck
        , train_data_heiniken
        , train_data_imperio
        , train_data_kaiser
        , train_data_original
        , train_data_skol_beats_gt
        , train_data_skol_beats_secret
        , train_data_skol_beats_senses
        , train_data_skol
        , train_data_sol
        , train_data_spaten
        , train_data_stella_artrois
    )
)

In [ ]:
train_data_y = to_categorical(
    np.concatenate(
        (
              np.zeros(len(train_data_amstel)) # * 0
            , np.ones(len(train_data_bavaria)) # * 1
            , np.ones(len(train_data_bohemia)) * 2
            , np.ones(len(train_data_brahma)) * 3
            , np.ones(len(train_data_colorado_appia_garrafa)) * 4
            , np.ones(len(train_data_colorado_appia_lata)) * 5
            , np.ones(len(train_data_colorado_indica_garrafa)) * 6
            , np.ones(len(train_data_colorado_indica_lata)) * 7
            , np.ones(len(train_data_coronita_long_neck)) * 8
            , np.ones(len(train_data_heiniken)) * 9
            , np.ones(len(train_data_imperio)) * 10
            , np.ones(len(train_data_kaiser)) * 11
            , np.ones(len(train_data_original)) * 12
            , np.ones(len(train_data_skol_beats_gt)) * 13
            , np.ones(len(train_data_skol_beats_secret)) * 14
            , np.ones(len(train_data_skol_beats_senses)) * 15
            , np.ones(len(train_data_skol)) * 16
            , np.ones(len(train_data_sol)) * 17
            , np.ones(len(train_data_spaten)) * 18
            , np.ones(len(train_data_stella_artrois)) * 19
        )
    )
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( train_data_X, train_data_y, test_size=0.3, random_state=42 )

# Modelagem e compilação

In [ ]:
learning_rate = 1e-4
# ada_belief = AdaBeliefOptimizer(learning_rate=learning_rate)
ada_belief = Adam(learning_rate=learning_rate)

In [ ]:
reset_seeds()

model = Sequential()

# Extração de caracteristicas
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D( 64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D( 32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Achatamento
model.add(Flatten())

# classificadores
model.add(Dense(128, activation='relu'))
model.add(Dense( 64, activation='relu'))
# model.add(Dense(  3, activation='sigmoid'))
model.add(Dense(  n_neuronios_saida, activation='softmax'))

In [ ]:
model.compile(optimizer=ada_belief, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Execução do treino do modelo

In [ ]:
%%time
with mlflow.start_run(run_name='cnn_img_beer'):
  reset_seeds()
  hist = model.fit(X_train, y_train, epochs=25, validation_split=0.2)
  model.evaluate(X_test, y_test)

# Predição com o modelo treinado

In [ ]:
# model.predict(X_test).argmax(axis=-1)

In [ ]:
from IPython.display import Image

def predic_img(i):

  image_path = f'{datasetPath}/test/{i}.jpg'
  
  image_original = tf.keras.utils.load_img(
      image_path,
      grayscale=False, 
      color_mode='rgb', 
      target_size=None, 
      interpolation='nearest'
  )

  image_resized = image_original.resize(size)

  image_prepared = np.expand_dims(image_resized, axis=0)

  prediction = model.predict(image_prepared)

  display(Image(image_path, width=50, height=50))

  n_pos = prediction.argmax(axis=-1)[0]
  
  print(
      'Prediction for item:', i
      , '- position:', n_pos
      , ' - label:', labels_index[n_pos]
      , '\n\n'
  )

In [ ]:
# predic_img('0')

In [ ]:
# for i in range(0, 20):
#   predic_img(i)  

# Salvar o modelo no google drive

In [ ]:
model.save('/content/drive/MyDrive/PucMinas/projeto_integrado/' + 'cnn_trained_model_beer.h5')